# Problema de flujo con oferta y demanda

Otra de las aplicaciones de este paquete es resolver problemas de flujo con oferta y demanda, este tipo de problemas es muy similar al problema original de flujo máximo pero ahora los nodos cuentan con un parámetro extra, una oferta o una demanda, es decir, un nodo origen debería tener una capacidad de oferta y un nodo destino debería tener una capacidad de demanda. 

Ahora el problema a resolver es encontrar un flujo que satisfaga la demanda, podemos hallar una solución para este problema con el algoritmo de Ford-Fulkerson, transformando la red de tal manera que la solución del problema de flujo máximo nos de la solución del problema de flujo con oferta y demanda.

Para poder hacer esta transformación necesitamos:

- Agregar un nodo de origen $s$ y arcos desde él a cada nodo origen original $f_i$ con capacidad $p_i$ donde $p_i$ es la oferta de cada nodo origen original.
    
- Agregar un nodo destino $t$ y arcos de todas los nodos destino originales $v_i$ a $t$ con capacidad $d_i$ donde $d_i$ es la demanda de cada nodo destino original.

Ahora con esta nueva red, existe un flujo que satisface la demanda si y solo si:
    
$$\text{Flujo Máximo} = \sum_{i \in V} d_i$$

Si ocurre lo anterior, la solución del problema de flujo máximo nos daría la respuesta de cuantas unidades enviar por cada arco para satisfacer la demanda, es decir, obtendriamos la solución del problema de flujo con oferta y demanda.

## Ejemplo 1:

Usaremos el ejemplo de la siguiente imagen:

![Flujo](../img/ford_fulkerson11.png)

En (a) observamos los nodos originales, estos tienen un número en su interior que representa la oferta y la demanda, los nodos blancos representan los nodos origen, por lo que el número representa una oferta y los nodos grises son nodos destino y estos tienen una demanda que cumplir. Observamos 5 arcos con un número que representa la capacidad de cada arco.

En (b) observamos la creación de dos nodos extra, $s^*$ y $t^*$, que serán nuestros nuevos nodos origen y destino respectivamente, también se crearon nuevos arcos con capacidad igual a la oferta o demanda del nodo al que se conectan.

En (c) observamos la solución de este problema, la solución del problema de flujo máximo es $6$ y es igual a la demanda de nuestra red, por lo que también se resolvió el problema de flujo con oferta y demanda.

## Implementación:

Primero resolveremos el problema con el paquete *ffmaxflow*

In [14]:
import ffmaxflow as ff

In [49]:
red = ff.create_flow_network()

In [50]:
red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)

In [51]:
red.create_edge('s', 'd', 3)
red.create_edge('s', 'a', 3)

red.create_edge('a', 'c', 3)
red.create_edge('a', 'd', 3)

red.create_edge('d', 'c', 3)
red.create_edge('d', 'b', 2)

red.create_edge('b', 't', 4)

red.create_edge('c', 't', 2)
red.create_edge('c', 'b', 2)

In [52]:
#%%time
red.MaxFlow()

CPU times: user 50 µs, sys: 63 µs, total: 113 µs
Wall time: 117 µs


6

Ahora resolveremos con *ffmaxc* y comprobaremos que los resultados coinciden.

In [15]:
import ffmaxc as ffc

red = ffc.create_flow_network()

red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)

red.create_edge('s', 'd', 3)
red.create_edge('s', 'a', 3)

red.create_edge('a', 'c', 3)
red.create_edge('a', 'd', 3)

red.create_edge('d', 'c', 3)
red.create_edge('d', 'b', 2)

red.create_edge('b', 't', 4)

red.create_edge('c', 't', 2)
red.create_edge('c', 'b', 2)

#%%time
red.MaxFlow()

6.0

 Ahora usaremos el paquete networkx para recuperar el flujo de cada arco y así obtener la solución para el problema de demanda.

In [6]:
import networkx as nx

G = nx.DiGraph()

G.add_edge('s', 'd', capacity=3)
G.add_edge('s', 'a', capacity=3)

G.add_edge('a', 'c', capacity=3)
G.add_edge('a', 'd', capacity=3)

G.add_edge('d', 'c', capacity=3)
G.add_edge('d', 'b', capacity=2)

G.add_edge('b', 't', capacity=4)

G.add_edge('c', 'b', capacity=2)
G.add_edge('c', 't', capacity=2)

#%%time
flow_value, flow_dict = nx.maximum_flow(G, 's', 't')

flow_dict

{'s': {'d': 3, 'a': 3},
 'd': {'c': 1, 'b': 2},
 'a': {'c': 3, 'd': 0},
 'c': {'b': 2, 't': 2},
 'b': {'t': 4},
 't': {}}

Observamos que nos da una solución distinta a la de la imagen, es decir, en el problema de flujo con oferta y demanda la solución no es única. Un siguiente paso podría ser agregar un costo a cada arco para así poder encontrar la solución optima.

## Tiempos de ejecución usando timeit

Mediremos el tiempo de ejecución de nuestras dos implementaciones y de *networkx* usando *timeit*, repitiendo el proceso 100,000 veces.

In [7]:
from timeit import timeit

Primera iteración del paquete:

In [12]:
setup_code = """import ffmaxflow as ff
import numpy as np
import pandas as pd
    
"""

statement = """
red = ff.create_flow_network()

red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)

red.create_edge('s', 'd', 3)
red.create_edge('s', 'a', 3)

red.create_edge('a', 'c', 3)
red.create_edge('a', 'd', 3)

red.create_edge('d', 'c', 3)
red.create_edge('d', 'b', 2)

red.create_edge('b', 't', 4)

red.create_edge('c', 't', 2)
red.create_edge('c', 'b', 2)
            
red.MaxFlow()  
            
"""

print (timeit(setup = setup_code, stmt = statement, number = 100000))

8.862846198000625


Reimplementación del código:

In [10]:
setup_code = """import ffmaxc as ffc
import numpy as np
import pandas as pd
    
"""

statement = """
red = ffc.create_flow_network()

red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)

red.create_edge('s', 'd', 3)
red.create_edge('s', 'a', 3)

red.create_edge('a', 'c', 3)
red.create_edge('a', 'd', 3)

red.create_edge('d', 'c', 3)
red.create_edge('d', 'b', 2)

red.create_edge('b', 't', 4)

red.create_edge('c', 't', 2)
red.create_edge('c', 'b', 2)
            
red.MaxFlow()  
            
"""

print (timeit(setup = setup_code, stmt = statement, number = 100000))

6.139643886002887


Usando *networkx*:

In [11]:
setup_code = """import networkx as nx
import numpy as np
import pandas as pd
"""

statement = """
G = nx.DiGraph()

G.add_edge('s', 'd', capacity=3)
G.add_edge('s', 'a', capacity=3)

G.add_edge('a', 'c', capacity=3)
G.add_edge('a', 'd', capacity=3)

G.add_edge('d', 'c', capacity=3)
G.add_edge('d', 'b', capacity=2)

G.add_edge('b', 't', capacity=4)

G.add_edge('c', 'b', capacity=2)
G.add_edge('c', 't', capacity=2)

#%%time
flow_value, flow_dict = nx.maximum_flow(G, 's', 't')

flow_dict
            
"""

print (timeit(setup = setup_code, stmt = statement, number = 100000))

23.07639731299787


## Ejemplo 2:

Usaremos la siguiente red para el ejemplo 2:

![Inicial](../img/inicialoferta.png)

Agregando los arcos y nodos necesarios la implementación del código queda de la siguiente manera:

In [19]:
red = ff.create_flow_network()

red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)
red.create_vertex('e', False, False)
red.create_vertex('f', False, False)

red.create_edge('s', 'a', 8)
red.create_edge('s', 'c', 7)
red.create_edge('s', 'b', 6)


red.create_edge('a', 'd', 6)
red.create_edge('a', 'e', 7)

red.create_edge('b', 'd', 7)
red.create_edge('b', 'f', 9)

red.create_edge('c', 'a', 10)
red.create_edge('c', 'd', 3)

red.create_edge('d', 't', 10)

red.create_edge('e', 'b', 4)
red.create_edge('e', 'f', 4)

red.create_edge('f', 't', 11)

#%%time
red.MaxFlow()

21

In [20]:
import ffmaxc as ffc

red = ffc.create_flow_network()

red.create_vertex('s', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)
red.create_vertex('e', False, False)
red.create_vertex('f', False, False)

red.create_edge('s', 'a', 8)
red.create_edge('s', 'c', 7)
red.create_edge('s', 'b', 6)


red.create_edge('a', 'd', 6)
red.create_edge('a', 'e', 7)

red.create_edge('b', 'd', 7)
red.create_edge('b', 'f', 9)

red.create_edge('c', 'a', 10)
red.create_edge('c', 'd', 3)

red.create_edge('d', 't', 10)

red.create_edge('e', 'b', 4)
red.create_edge('e', 'f', 4)

red.create_edge('f', 't', 11)

red.MaxFlow()

21.0

In [22]:
G = nx.DiGraph()

G.add_edge('s', 'a', capacity=8)
G.add_edge('s', 'c', capacity=7)
G.add_edge('s', 'b', capacity=6)


G.add_edge('a', 'd', capacity=6)
G.add_edge('a', 'e', capacity=7)

G.add_edge('b', 'd', capacity=7)
G.add_edge('b', 'f', capacity=9)

G.add_edge('c', 'a', capacity=10)
G.add_edge('c', 'd', capacity=3)

G.add_edge('d', 't', capacity=10)

G.add_edge('e', 'b', capacity=4)
G.add_edge('e', 'f', capacity=4)

G.add_edge('f', 't', capacity=11)

#%%time
flow_value, flow_dict = nx.maximum_flow(G, 's', 't')

flow_dict

{'s': {'a': 8, 'c': 7, 'b': 6},
 'a': {'d': 6, 'e': 6},
 'c': {'a': 4, 'd': 3},
 'b': {'d': 1, 'f': 7},
 'd': {'t': 10},
 'e': {'b': 2, 'f': 4},
 'f': {'t': 11},
 't': {}}

El flujo máximo es 21 y la demanda de los nodos es 21, por lo que existe solución para el problema de flujo con oferta y demanda. En la siguiente gráfica podemos visualizar la solución.

![FlujoMax](../img/soloferta.png)